In [1]:
script_path = getwd()                                # %exclude_jupyterlab%

In [2]:
script_path                                          # %exclude_jupyterlab%

[1] "/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/scripts/rna"

In [3]:
# system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
#         '/sc_singleR_opt.R --genome="hg38" --markerList="/Volumes/ccrsf-ifx/Software/scripts/bin/currentsnake/single_cell/gene_lists/human_gene_list.csv" --outdir="/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/singleR" --rds="/Volumes/ccrsf-static/Analysis/xies4/github_repos/pipeline_dev_test/test_dir/seur_10x_cluster_object.rds"'),  # %exclude_jupyterlab%
#        intern = T) # %exclude_jupyterlab%

In [4]:
# system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
#         '/sc_singleR_opt.R --genome="hg38" --markerList="/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/data/marker_genelist/human_gene_list.csv" --outdir="/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/test_singleR/" --rds="/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/split_pipe_comb/1V232R/seurat/seur_10x_cluster_object.rds"'),  # %exclude_jupyterlab%
#        intern = T) # %exclude_jupyterlab%

In [5]:
system(paste0('Rscript ', script_path,                # %exclude_jupyterlab%
        '/sc_singleR_opt.R --genome="hg38" --markerList="/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/data/marker_genelist/human_gene_list.csv" --outdir="/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/test_singleR/" --rds="/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039851_16sclibParse_080625/Analysis/split_pipe_comb/1V231R/seurat/seur_10x_cluster_object.rds"'),  # %exclude_jupyterlab%
       intern = T) # %exclude_jupyterlab%

character(0)

In [6]:
opt = readRDS("/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/test_singleR/opt.rds")         # %exclude_jupyterlab%

In [7]:
opt

$rds
[1] "/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039851_16sclibParse_080625/Analysis/split_pipe_comb/1V231R/seurat/seur_10x_cluster_object.rds"

$sampleid
[1] "scRNA"

$genome
[1] "hg38"

$markerList
[1] "/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/workflow/data/marker_genelist/human_gene_list.csv"

$outdir
[1] "/mnt/ccrsf-static/singlecell_projects/CarolinaBarillasMury_CS039906_16Parselib_081125/Analysis/test_singleR/"

$help
[1] FALSE

In [8]:
system(paste0("mkdir -p ", opt$outdir))
setwd(opt$outdir) 

In [9]:
library(Seurat)
library(SingleR)
library(celldex)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    ro

In [10]:
seur <- readRDS(opt$rds)

In [11]:
DefaultAssay(seur) = "RNA"
sce <- as.SingleCellExperiment(seur)
range(colSums(counts(sce)))
DefaultAssay(seur) = "SCT"

Warning message:
“Layer ‘data’ is empty”
Warning message:
“Layer ‘scale.data’ is empty”


[1]  30 473

In [12]:
sce <- sce[, colSums(counts(sce)) > 0]

In [13]:
range(colSums(counts(sce)))

[1]  30 473

In [14]:
sce <- scater::logNormCounts(sce)

In [15]:
geneList <- read.csv(opt$markerList, header=FALSE, row.names=1, stringsAsFactors = FALSE)

In [16]:
## plotScoreDistribution: use this plot to check that outlier detection in pruneScores() behaved sensibly. 
## Labels with especially low deltas may warrant some additional caution in their interpretation.
generatePlots <- function(name, pred, seur, markers){
  saveRDS(pred, paste0('pred_', name, '.rds'))
  #pdf(paste0('heatmap_', name, '.pdf'))
  #print(plotScoreHeatmap(pred, clusters=sce$res.0.1))
  #dev.off()
  pdf(paste0('ScoreDistribution_', name, '.pdf'), height=15)
  # plotScoreDistribution deprecated
  #print(plotScoreDistribution(pred, show = "delta.med", ncol = 3, show.nmads = 3))
  tryCatch({
    p <- plotDeltaDistribution(pred, show="delta.med", ncol=3)
    print(p)
  }, error=function(e) message("Skipping delta distribution plot: ", conditionMessage(e)))
  dev.off()
  seur@meta.data[name] <- ''
  seur@meta.data[rownames(pred),name] <- pred$pruned.labels
  Idents(seur) <- seur@meta.data[,name]
  png(paste0("UMAP_", name, ".png"))
  print(DimPlot(seur))
  dev.off()
  png(paste0("TSNE_", name, ".png"))
  print(DimPlot(seur, reduction='tsne'))
  dev.off()
  dir.create("gene_list_plots")
  for (i in 1:dim(markers)[1]) {
    genes.use <- as.character(markers[row.names(markers)[i],])
    numGenes <- length(genes.use[genes.use %in% row.names(seur)])
    if (sum(genes.use %in% row.names(seur)) > 1) {
      png(paste0('gene_list_plots/ViolinPlot_', name, '_', row.names(geneList)[i],'.png'), height = 5, width = (length(genes.use[genes.use %in% row.names(seur)])+1)*5, units='in', res=300)
      print(suppressWarnings(VlnPlot(seur, genes.use[genes.use %in% row.names(seur)], ncol=numGenes)))
      dev.off()
    }
  }
  return(seur)
}

In [17]:
generateMarkerPlots <- function(seur, markers) {
  dir.create("gene_list_plots")
  for (i in 1:dim(markers)[1]) {
    genes.use <- as.character(markers[row.names(markers)[i],])
    numGenes <- length(genes.use[genes.use %in% row.names(seur)])
    if (sum(genes.use %in% row.names(seur)) > 1) {
      png(paste0('gene_list_plots/FeaturePlot_', row.names(geneList)[i],'.png'), height = 5, width = (length(genes.use[genes.use %in% row.names(seur)])+1)*5, units='in', res=300)
      print(suppressWarnings(FeaturePlot(seur, genes.use[genes.use %in% row.names(seur)], ncol=numGenes)))
      dev.off()
    }
  }
}

In [ ]:
# The objects returned by functions:  ImmGenData, MouseRNAseqData, 
# HumanPrimaryCellAtlasData and BlueprintEncodeData are saved under 
# /home/docker/. So you only need to load the RDS files directly. 
if (startsWith(opt$genome, "mm10") | startsWith(opt$genome, "mm39")) {
  #immgen <- ImmGenData()# - mouse
  #mouserna <- MouseRNAseqData()# - mouse
  immgen <- readRDS("/home/docker/ImmGenData.rds")
  mouserna <- readRDS("/home/docker/MouseRNAseqData.rds")
  pred.multi <- SingleR(test = sce,
      ref = list(IG=immgen, MRNA=mouserna),
      labels = list(immgen$label.main, mouserna$label.main))
  seur <- generatePlots('immgen_mourserna', pred.multi, seur, geneList)
  temp <- table(pred.multi$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_Immgen-MouseRNA_general.csv', row.names=FALSE)

  pred.immgen <- SingleR(test = sce,
      ref = immgen, labels = immgen$label.main)
  seur <- generatePlots('immgen', pred.immgen, seur, geneList)
  temp <- table(pred.immgen$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_Immgen_general.csv', row.names=FALSE)

  pred.mouserna <- SingleR(test = sce,
      ref = mouserna, labels = mouserna$label.main)
  seur <- generatePlots('mourserna', pred.mouserna, seur, geneList)
  temp <- table(pred.mouserna$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_MouseRNA_general.csv', row.names=FALSE)

  write.csv(seur@meta.data[,c('immgen_mourserna', 'immgen', 'mourserna')], 'annotations_compiled_barcode.csv', quote=FALSE)
} else if (startsWith(opt$genome, "hg38") | startsWith(opt$genome, "hg19")) {
  #hpca <- HumanPrimaryCellAtlasData()
  #blueprint <- BlueprintEncodeData()
  hpca <- readRDS("/home/docker/HumanPrimaryCellAtlasData.rds")
  blueprint <- readRDS("/home/docker/BlueprintEncodeData.rds")
  pred.multi <- SingleR(test = sce,
      ref = list(BP=blueprint, HPCA=hpca),
      labels = list(blueprint$label.main, hpca$label.main))
  seur <- generatePlots('blueprintencode_hpca', pred.multi, seur, geneList)
  temp <- table(pred.multi$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_BlueprintENCODE-HPCA_general.csv', row.names=FALSE)

  pred.blueprint <- SingleR(test = sce,
      ref = blueprint, labels = blueprint$label.main)
  seur <- generatePlots('blueprintencode', pred.blueprint, seur, geneList)
  temp <- table(pred.blueprint$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_BlueprintENCODE_general.csv', row.names=FALSE)

  pred.hpca <- SingleR(test = sce,
      ref = hpca, labels = hpca$label.main)
  seur <- generatePlots('hpca', pred.hpca, seur, geneList)
  temp <- table(pred.hpca$pruned.labels)
  write.csv(sort(temp, decreasing=TRUE), 'annotations_HPCA_general.csv', row.names=FALSE)

  write.csv(seur@meta.data[,c('blueprintencode_hpca', 'blueprintencode', 'hpca')], 'annotations_compiled_barcode.csv', quote=FALSE)
}

suppressWarnings(generateMarkerPlots(seur, geneList))
saveRDS(seur, 'seur_10x_cluster_singler.rds')


In [ ]:
sessionInfo()

In [ ]:
notebook_prefix = "sc_singleR"                                                        # %exclude_jupyterlab%
notebook_name = paste0(notebook_prefix, ".ipynb")                                     # %exclude_jupyterlab%
notebook_r = paste0(script_path, "/", paste0(notebook_prefix, ".r"))                  # %exclude_jupyterlab%
notebook_path = paste0(script_path, "/", notebook_name)                               # %exclude_jupyterlab%
opt_name = paste0(script_path, "/", sub(".ipynb", "_opt.R", notebook_name))           # %exclude_jupyterlab%
output = paste0(script_path, "/", sub(".ipynb", ".prod.R", notebook_name))            # %exclude_jupyterlab%
cmd1 = paste0("jupyter nbconvert --to script --output ",                              # %exclude_jupyterlab%
             notebook_prefix, ' ', notebook_path, "> /dev/null 2>&1 ")                # %exclude_jupyterlab%
cmd1                                                                                  # %exclude_jupyterlab%
system(cmd1, intern = TRUE)                                                           # %exclude_jupyterlab%

In [ ]:
cmd2 = paste0('cat -s ', opt_name, ' ', notebook_r,                                   # %exclude_jupyterlab%
             ' |grep -v exclude_jupyterlab > ', output,  ' 2>&1')                     # %exclude_jupyterlab%
cmd2                                                                                  # %exclude_jupyterlab%
system(cmd2, intern = T)                                                              # %exclude_jupyterlab%
system(paste0("rm ", notebook_r))                                                     # %exclude_jupyterlab%  